# Open Street Map data

## Project Overview
The aim of this project is to choose any area of the world in https://www.openstreetmap.org and use data wrangling techniques, such as assessing the quality of the data for validity, accuracy, completeness, consistency and uniformity, to clean the OpenStreetMap data for that part of the world. Once the data has been cleaned SQL will will be used to query and aggregate the data.

### OSM Dataset
The area selected for this project is Dublin city in Ireland. Dublin is the capital city of Ireland with a population of 1,347,359 people. I chose Dublin as I've lived there for nearly 10 years and am quite familiar with the area. 

* Location: Dublin, Ireland
* [OpenStreetMap URL](https://www.openstreetmap.org/export#map=11/53.3549/-6.2512)
* [MapZen URL](https://mapzen.com/data/metro-extracts/metro/dublin_ireland/) 

## Data Overview

Let's get an idea of what the top level tags are in the OSM file. Since the file is quite large iterative parsing will be used to process the map file and find out what tags are there, as well as how many, to get the feeling on how much of which data you can expect to have in the map.

In [1]:
OSMFILE = "dublin_ireland.osm"

In [2]:
import top_level_tags
top_level_tags.count_tags(OSMFILE)

{'bounds': 1,
 'member': 91716,
 'nd': 2028541,
 'node': 1469711,
 'osm': 1,
 'relation': 4989,
 'tag': 1060747,
 'way': 269763}

The file is 369.4 MB and there are nearly 5,000,000 top level tags. 

### Check for potential problems in tags

Let's explore the data a bit more. Before processing the data and adding it into a database, we can check the
"k" value for each "tag" tag and see if there are any potential problems.

We can get a count of each of four tag categories in a dictionary:
* "lower", for tags that contain only lowercase letters and are valid,
* "lower_colon", for otherwise valid tags with a colon in their names,
* "problemchars", for tags with problematic characters, and
* "other", for other tags that do not fall into the other three categories.

In [3]:
import tagtype
tagtype.process_map(OSMFILE)

{'lower': 673577, 'lower_colon': 342839, 'other': 44331, 'problemchars': 0}

### Number of contributors

OpenStreetMap consists of data contributed by multiple people. Each piece of data in the OSMFILE is accompanied by the user_id of the person who entered it. We can find out how many people have contributed towards the data the makes up the map of Dublin:

In [4]:
import users
users.number_of_users(OSMFILE)

Number of unique contributors: 1602


## Data Auditing

The following steps will be taken to audit the OSMFILE:
1. Create a variable, 'mapping', that will replace incorrect or inconsistent entries with appropriate names/formating. Only problems found in this OSMFILE will use mapping rather than a generalized solution, since that may and will depend on the particular area being audited.
2. Write a function to actually fix the street name. The function takes a string with street name as an argument and should return the fixed version.

### 1. Fix street names

In [13]:
%%capture
import audit
audit.audit_street(OSMFILE)

Although plenty of extra street names show up that weren't in the expected list, most of them are less common street names that are acceptable. There are a few abbreviated street names and some spelling mistakes that can be fixed using mapping.

In [6]:
audit.update_street_name(OSMFILE)

Strand Rd. => Strand Road
Upper Gardiner St. => Upper Gardiner Street
Charlemont St. => Charlemont Street
Old Dublin Roafd => Old Dublin Road
Griffith Ave => Griffith Avenue
First Ave => First Avenue
Spruce Ave => Spruce Avenue
The Rise,Belgard heights => The Rise,Belgard Heights
Ballinclea heights => Ballinclea Heights
Charlestown Shopping Cente => Charlestown Shopping Centre
library square => library Square
Francis St => Francis Street
Woodview Heichts => Woodview Heights
Oak Ridge Cres => Oak Ridge Crescent
Suffolk street => Suffolk Street
Earl street => Earl Street
Grafton street => Grafton Street
O'Reilly Aveune => O'Reilly Avenue
New market hall => New market Hall
St Johns Rd => St Johns Road
Novara road => Novara Road
Warner's lane => Warner's Lane
Hanbury lane => Hanbury Lane


'Clonkeen'

### 2. Fix Eircodes (Postal codes)

In [12]:
%%capture
import eircodes
eircodes.audit(OSMFILE)

The acceptable format for Eircodes is XXX XXXX. Some of the Eircodes are missing the space in the middle so we'll fix this. 

Ireland recently introduced Eircodes which each represent a single building. There are some post codes that are still using the old convention of postcode which was listed as Dublin and a number to indicate the region of Dublin, e.g "Dublin 2". The numbers listed at the top are likely from the older system of postcodes where it was common to either list a postcode in the format "Dublin 2", "D2" or just "2" to specify the region of Dublin. 

These old post codes can't be easily translated to Eircodes. The old convention represented a region within Dublin but the new Eircodes represent specific buildings. So for each of the old post codes there are now hundreds of unique Eircodes in it's place. To convert over to the new system would require time consuming manual work of pin pointing each of the specific building and finding out what the newly assigned Eircode is for that building. So, we will just stick to tidying up the formatting for the new Eircodes that have been entered.

In [8]:
eircodes.update_eircode(OSMFILE)

D01X2P2 => D01 X2P2
D15KPW7 => D15 KPW7
D02X285 => D02 X285
D05N7F2 => D05 N7F2
D08P 89W => D08 P89W
D6WXK28 => D6W XK28
d09 f6x0 => D09 F6X0
D09VY19 => D09 VY19


'D01 NW14'

## Preparing Data for Database

After auditing is complete the next step is to prepare the data to be inserted into a SQL database. To do so we will parse the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to .csv files.  These csv files can then easily be imported to a SQL database as tables. 

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

The data.py file generates the following csv files:
* nodes_tags.csv
* nodes.csv 
* ways_nodes.csv
* ways_tags.csv
* ways.csv

We can import the csv files into pandas to get a quick look at the data contained within the csv files:

In [4]:
import pandas as pd
df_node = pd.read_csv('nodes.csv')
df_nodes_tags = pd.read_csv('nodes_tags.csv')
df_ways = pd.read_csv('ways.csv')
df_ways_tags = pd.read_csv('ways_tags.csv')
df_ways_nodes = pd.read_csv('ways_nodes.csv')

In [2]:
df_node.head()

,id,lat,lon,user,uid,version,changeset,timestamp
0,384519,53.376342,-6.369919,Joe E,114310,3,5277755,2010-07-21T10:37:10Z
1,385708,53.388803,-6.353715,mackerski,6367,4,24196387,2014-07-17T08:22:15Z
2,385740,53.390833,-6.379218,mackerski,6367,1,3703,2007-04-01T01:02:49Z
3,385752,53.388140,-6.379561,mackerski,6367,5,19334388,2013-12-08T02:00:11Z
4,385807,53.406947,-6.301549,ManAboutCouch,51337,7,5853138,2010-09-23T11:14:39Z


In [5]:
df_nodes_tags.head()

,id,key,value,type
0,385740,created_by,YahooApplet 1.0,regular
1,385935,highway,traffic_signals,regular
2,386265,highway,traffic_signals,regular
3,386490,created_by,YahooApplet 1.0,regular
4,389630,highway,crossing,regular


In [6]:
df_ways.head()

,id,user,uid,version,changeset,timestamp
0,2110697,VictorIE,2008037,9,47167020,2017-03-26T04:09:12Z
1,2294449,VictorIE,2008037,12,52193319,2017-09-19T20:55:52Z
2,2642220,Throwaway241,5308354,5,48714756,2017-05-15T22:29:50Z
3,2684343,VictorIE,2008037,5,21820716,2014-04-20T18:35:38Z
4,2684725,mackerski,6367,7,24748711,2014-08-14T15:56:37Z


In [7]:
df_ways_tags.head()

,id,key,value,type
0,2110697,name,St. Stephen's Green,regular
1,2110697,lanes,1,regular
2,2110697,oneway,yes,regular
3,2110697,highway,unclassified,regular
4,2110697,en,St. Stephen's Green,name


In [8]:
df_ways_nodes.head()

,id,node_id,position
0,2110697,4009885977,0
1,2110697,389664,1
2,2110697,26165089,2
3,2294449,9100863,0
4,2294449,1998326263,1


## Data Exploration

This section will explore the data within the CSV files using SQL. The data was imported into 5 seperate tables using SQLite by entering ".mode csv" followed by importing the csv file. This was done on the sample.osm file as the original file was too large for creating csv files.

### File sizes:
* dublin_ireland.osm - 369.2 MB
* sample_osm - 10.6  MB
* nodes.csv - 4.2 MB
* nodes_tags.csv - 201 KB 
* ways.csv - 463 KB
* ways_tags.csv - 922 KB
* ways_nodes.csv - 2.1 MB

## Number of nodes and ways:

### Number of nodes:

In [ ]:
sqlite> SELECT COUNT(*) FROM nodes;

**Output:** 41992

### Number of ways:

In [ ]:
sqlite> SELECT COUNT(*) FROM ways;

**Output:** 7708 

## Conributing users
OpenStreetMap is acollabrative project so many users are involved in enetering the data that is used. We can query the database created in order to find out some more information about the number of users involved in creating the map for Dublin.


### Number of Unique Users:

In [ ]:
sqlite> SELECT COUNT(DISTINCT(t.uid))          
        FROM (
            SELECT uid FROM nodes UNION ALL SELECT uid FROM ways
            ) t;

**Output:** There are 619 unique users.

### Number of users with one contribution:

In [ ]:
sqlite> SELECT COUNT(*) 
        FROM(
            SELECT t.user, COUNT(*) as num
            FROM (
                SELECT user FROM nodes UNION ALL SELECT user FROM ways) t
            GROUP BY t.user
        HAVING num=1) t2;

**Output:** 206 users have made one contribution.

### Top 5 users with the most contributions:

In [ ]:
sqlite> SELECT t.user, COUNT(*) as num
        FROM (
            SELECT user FROM nodes UNION ALL SELECT user FROM ways
            ) t
        GROUP BY t.user
        ORDER BY num DESC
        LIMIT 5;

**Output:** The top 5 users with the most contributions are:
* "Nick Burrett",6220
* brianh,5840
* Dafo43,5635
* mackerski,5609
* VictorIE,4874

## Number of schools:

In [ ]:
sqlite> SELECT COUNT(*) AS num
        FROM nodes_tags
        WHERE nodes_tags.value = 'school';

**Output:** There are 2 schools listed.

## Number of amenities:

In [ ]:
sqlite> SELECT value, COUNT(*) AS num
        FROM nodes_tags
        WHERE nodes_tags.key = 'amenity'
        GROUP BY value
        ORDER BY num DESC 
        LIMIT 10;

**Output:** The top 10 amenities are:
* bench,22
* restaurant,21
* cafe,20
* bicycle_parking,18
* pub,18
* post_box,15
* fast_food,9
* bank,8
* waste_basket,8
* atm,7

## Religion:
Ireland is a religious country with a largely Christian population so it'll be interesting to see the ratio of different religions within Dublin. 

In [ ]:
sqlite> SELECT nodes_tags.value, COUNT(*) as num
        FROM nodes_tags 
            JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') t
            ON nodes_tags.id=t.id
        WHERE nodes_tags.key='religion'
        GROUP BY nodes_tags.value
        ORDER BY num DESC;
        


**Output:** Strangely enough, this isn't showing an output so I'll check to see how many places of worship appear in the csv file:  

In [ ]:
SELECT COUNT(*) AS num
        FROM nodes_tags
        WHERE nodes_tags.value = 'place_of_worship';

This shows that there are 0 places of worship recorded in the csv file. Since the csv file was generated from a small sample of the map of Dublin it is not entirely represntative of all of Dublin.

## 5. Additional ideas

Suggestions for improving the data accuracy in OpenStreetMap include:
* Make all letters in postcodes uppercase - If OSM stored all letters in postcodes in uppercase there would be a bit more consistency. 
* There should be documentation on standard practices, e.g. for phone numbers whether to use brackets, dashes and spaces. Some fields had underscores filled in as the user who entered the data probably didn't know that bit of information. There could also check to make sure that just numbers have been entered, no special characters, except + which is often used in area codes.  
* Additionally OSM could have guidelines for whether addresses contain full names or abbreviated name for example, "Avenue" or "Ave." 

Anticipated problems for suggestions:
* A contributor may want to add a new building but not know the phone number. Forcing validation, a user may just entry a random list of numbers with the same format as a phone number. This would cause issues later on when wrangling the data as it would be harder to detect incorrect phone numbers.
* OpenStreetMap relies on open source data, so enforcing stricter validation may lead to less people contributing towards he data. There may be local users who want to input a small bit of information about their favourite coffee shop for example, but the might not know the post code. Stricter validation may lead to users being unable to upload small bits of information about places they know which could then lead to a much less complete set of data for OpenStreetMap to work with.

### Files

* top_level_tags -> Find out waht the top level tags are and how many of them there are.
* tagtype.py -> Check the "k" value for each "tag" tag and see if there are any potential problems.
* users.py -> Find out how many users contributed to the Dublin map.
* audit.py -> audit data for street names.
* eircodes.py -> audit data for eircodes.
* schema.py -> schemas to be generated.
* data.py -> Clean data and store in genrated csv files.